In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData, draw_matches

%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR)

# Test stereovision

In [ ]:
l_img, r_img = dataset.get_color_images(20)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(l_img)
ax2.imshow(r_img)

In [ ]:
left_matcher = cv2.StereoBM_create(numDisparities=16*9, blockSize=15)

# wls_filter = cv2.ximgproc.createDisparityWLSFilter(left_matcher)
# right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

left_matcher.setMinDisparity(0)
left_matcher.setSpeckleRange(8)
left_matcher.setSpeckleWindowSize(200)
left_matcher.setDisp12MaxDiff(1)
left_matcher.setUniquenessRatio(5)
left_matcher.setPreFilterCap(1)
left_matcher.setPreFilterSize(5)
left_matcher.setTextureThreshold(5)

l_img_gray = cv2.cvtColor(l_img, cv2.COLOR_RGB2GRAY)
r_img_gray = cv2.cvtColor(r_img, cv2.COLOR_RGB2GRAY)

l_disp = left_matcher.compute(l_img_gray, r_img_gray)
# r_disp = right_matcher.compute(r_img_gray, l_img_gray)

# wls_filter.setLambda(8000)
# wls_filter.setSigmaColor(8000)
# # wls_filter->filter(left_disp,left,filtered_disp,right_disp);
# wls_filter.filter(l_disp, l_img_gray, )

plt.figure(figsize=(15,9))
plt.imshow(l_disp)

https://answers.opencv.org/question/182049/pythonstereo-disparity-quality-problems/

In [ ]:
# window_size = 3
# min_disp = 1
# num_disp = 16*2
# stereo = cv2.StereoSGBM(
#     minDisparity=min_disp,
#     numDisparities=num_disp,
#     SADWindowSize=window_size,
#     uniquenessRatio=10,
#     speckleWindowSize=100,
#     speckleRange=32,
#     disp12MaxDiff=1,
#     P1=8*3*window_size**2,
#     P2=32*3*window_size**2,
#     fullDP=False
# )


left_matcher = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=16*9, 
    blockSize=15,
    P1=0,
    P2=0,
    disp12MaxDiff=1,
#     preFilterCap=1,
    uniquenessRatio=5,
    speckleWindowSize=200,
    speckleRange=8
)

# wls_filter = cv2.ximgproc.createDisparityWLSFilter(left_matcher)
# right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

l_img_gray = cv2.cvtColor(l_img, cv2.COLOR_RGB2GRAY)
r_img_gray = cv2.cvtColor(r_img, cv2.COLOR_RGB2GRAY)

l_disp = left_matcher.compute(l_img_gray, r_img_gray)
# r_disp = right_matcher.compute(r_img_gray, l_img_gray)

# wls_filter.setLambda(8000)
# wls_filter.setSigmaColor(8000)
# # wls_filter->filter(left_disp,left,filtered_disp,right_disp);
# wls_filter.filter(l_disp, l_img_gray, )

# plt.figure(figsize=(15,9))
# plt.imshow(l_disp)

disp = l_disp.astype(np.float32) / 16.0
plt.figure(figsize=(15,9))
plt.imshow(disp)

In [ ]:
# Output is always int16

Q_left = dataset.get_color_left_Q_matrix()
print(Q_left)
points = cv2.reprojectImageTo3D(disp, Q_left)

plt.figure(figsize=(15,9))
plt.imshow(points[:,:,2])

# points = points.reshape(-1, 3)
idxs = (disp > 0) & (points[:,:,2] < 100)
points = points[idxs]
colors = l_img[idxs]

print(colors.shape)
print(points.shape)

colors = ['#%02x%02x%02x' % (clr[0], clr[1], clr[2]) for clr in colors]

In [ ]:
fig = plt.figure(figsize=[20,9])
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], c=colors, marker='o')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')